In [ ]:
import gradio as gr
import requests
import json

# --- Constants ---
OLLAMA_URL = "https://evolving-sailfish-closing.ngrok-free.app"
MODEL_NAME = "qwen2.5-coder"

SYSTEM_PROMPT = {
    "role": "system",
    "content": (
        "You are a professional AI coding assistant. "
        "Be concise, technically accurate, and helpful. "
        "Use clean code, best practices, and include examples where appropriate."
    )
}

# --- Core Chat Logic ---

def create_new_chat():
    """Creates a new chat history with the system prompt."""
    return [SYSTEM_PROMPT.copy()]

def chat_with_ollama(user_message, history):
    """Streams a response from the Ollama API and updates the history."""
    history.append({"role": "user", "content": user_message})

    try:
        response = requests.post(
            f"{OLLAMA_URL}/api/chat",
            json={"model": MODEL_NAME, "messages": history, "stream": True},
            stream=True,
            timeout=120,
        )
        response.raise_for_status()

        assistant_reply = ""
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode("utf-8"))
                    if "message" in data and "content" in data["message"]:
                        token = data["message"]["content"]
                        assistant_reply += token
                        yield history + [{"role": "assistant", "content": assistant_reply}]
                    if data.get("done"):
                        break
                except json.JSONDecodeError:
                    print(f"Warning: Could not decode JSON line: {line}")
                    continue

        history.append({"role": "assistant", "content": assistant_reply})

    except requests.exceptions.RequestException as e:
        error_message = f"API Error: Could not connect to Ollama. Details: {e}"
        history.append({"role": "assistant", "content": error_message})
        yield history

# --- Helper Function for UI ---

def format_chat_for_display(history):
    """Filters out the system prompt for display in the chatbot UI."""
    return [m for m in history if m.get("role") != "system"]

# --- Gradio UI and Event Handlers ---

css = """
/* Force the radio buttons into a single vertical column */
.chat-list-radio .gr-form {
    display: flex;
    flex-direction: column !important;
    align-items: stretch !important;
}

/* Set a maximum width for the sidebar column */
#left-sidebar {
    max-width: 300px !important;
}
"""

with gr.Blocks(theme=gr.themes.Soft(), css=css) as demo:
    # State components to hold session data
    chat_sessions = gr.State([create_new_chat()])
    chat_names = gr.State(["Chat 1"])
    selected_chat_idx = gr.State(0)

    gr.Markdown("# Coding Assistant")

    with gr.Row(equal_height=False):
        # --- Left Sidebar ---
        with gr.Column(scale=0.5, min_width=200, elem_id="left-sidebar"):
            # Move the button to the top of the sidebar for better UI
            new_chat_btn = gr.Button("➕ New Chat", variant="secondary")

            gr.Markdown("### Chats")
            chat_list = gr.Radio(
                choices=["Chat 1"],
                value="Chat 1",
                label="Chat Sessions",
                show_label=False,
                interactive=True,
                elem_classes=["chat-list-radio"]
            )

        # --- Main Chat Area ---
        with gr.Column(scale=3.5):
            chatbot = gr.Chatbot(
                type="messages",
                label=f"Model: {MODEL_NAME}",
                bubble_full_width=False,
                height=500,
                render_markdown=True,
                latex_delimiters=[{"left": "$$", "right": "$$", "display": True}, {"left": "$", "right": "$", "display": False}],
            )
            with gr.Row():
                msg_textbox = gr.Textbox(
                    show_label=False,
                    placeholder="Ask a coding question...",
                    scale=7,
                    autofocus=True,
                )
                send_btn = gr.Button("Send", variant="primary", scale=1)

    # --- Event Handler Functions ---

    def add_new_chat(sessions, names):
        sessions.insert(0, create_new_chat())
        new_name = f"Chat {len(sessions)}"
        names.insert(0, new_name)
        new_idx = 0
        return (
            sessions, names, new_idx, [], gr.update(choices=names, value=names[0])
        )

    def switch_chat(choice, sessions, names):
        if not choice or choice not in names:
            idx = 0
        else:
            idx = names.index(choice)
        history = sessions[idx]
        return idx, format_chat_for_display(history)

    def handle_user_message(user_message, sessions, current_idx):
        if not user_message.strip():
            yield "", format_chat_for_display(sessions[current_idx]), sessions
            return

        history = sessions[current_idx]
        yield "", format_chat_for_display(history) + [{"role": "user", "content": user_message}], sessions

        stream = chat_with_ollama(user_message, history)
        for partial_history in stream:
            sessions[current_idx] = partial_history
            yield "", format_chat_for_display(partial_history), sessions

    # --- Connecting Handlers to Components ---

    new_chat_btn.click(
        fn=add_new_chat,
        inputs=[chat_sessions, chat_names],
        outputs=[chat_sessions, chat_names, selected_chat_idx, chatbot, chat_list]
    )

    chat_list.input(
        fn=switch_chat,
        inputs=[chat_list, chat_sessions, chat_names],
        outputs=[selected_chat_idx, chatbot]
    )

    submit_event = [send_btn.click, msg_textbox.submit]
    for event in submit_event:
        event(
            fn=handle_user_message,
            inputs=[msg_textbox, chat_sessions, selected_chat_idx],
            outputs=[msg_textbox, chatbot, chat_sessions],
            show_progress="hidden"
        )


if __name__ == "__main__":
    demo.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/layouts/column.py:59: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/layouts/column.py:59: UserWarning: 'scale' value should be an integer. Using 3.5 will cause issues.
  warnings.warn(
/tmp/ipython-input-2-912187176.py:107: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6ce64ba5f69f275841.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6ce64ba5f69f275841.gradio.live
